<a href="https://colab.research.google.com/github/eyalgur1/Asynchronous_ToA_Source_Localization/blob/main/phase2_vectorized.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### **Set Up Environment**

In [ ]:
import cvxpy as cp
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from scipy import sparse

In [ ]:
# only for first run, when running through Google Drive
from google.colab import drive
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import pickle

# Define a function to save all variables
def save_variables(filename):

    # Create a dictionary containing only the variables we want to save
    variables_to_save = {
        'trains': trains,
        'MGs': MGs,
        'TrainLoad': TrainLoad,
        'REG2TrainESS': REG2TrainESS,
        'TrainAtMG': TrainAtMG,
        'TrainCycle': TrainCycle,
        'GRIDprice': GRIDprice,
        'PVprod': PVprod,
        'T': T,
        'TrainESSdischargeMAX': TrainESSdischargeMAX,
        'TrainESSchargeMAX': TrainESSchargeMAX,
        'TrainESSchargeLoss': TrainESSchargeLoss,
        'TrainESSdischargeLoss': TrainESSdischargeLoss,
        'TrainESSmin': TrainESSmin,
        'TrainESScap': TrainESScap,
        'MGESSchargeMAX_ratio': MGESSchargeMAX_ratio,
        'MGESSdischargeMAX_ratio': MGESSdischargeMAX_ratio,
        'MGESSchargeLoss': MGESSchargeLoss,
        'MGESSdischargeLoss': MGESSdischargeLoss,
        'MGESSmin': MGESSmin,
        'MGESScapMAX': MGESScapMAX,
        'PVprod': PVprod,
        'PVcapMAX': PVcapMAX,
        'GRIDprod': GRIDprod,
        'GRIDprice': GRIDprice,
        'PVmaintCoeff': PVmaintCoeff,
        'MGESSmaintCoeff': MGESSmaintCoeff,
        'MG2TrainMAX': MG2TrainMAX,
        'PVcost': PVcost,
        'MGESScost': MGESScost,
        'CostsCoeff': CostsCoeff,
        'TrainLoad': TrainLoad,
        'REG2TrainESS': REG2TrainESS,
        'MGESSsocInit_ratio': MGESSsocInit_ratio,
        'TrainESSsocInit': TrainESSsocInit,
        'solver': solver,
        'GRID_MGs': GRID_MGs,
        'num_trains': num_trains,
        'num_MGs': num_MGs,
    }

    # Save the variables using pickle
    with open(filename, 'wb') as f:
        pickle.dump(variables_to_save, f)


def load_variables(filename):
    with open(filename, 'rb') as f:
        # Load the variables from the file
        loaded_variables = pickle.load(f)
        # Update the current global scope with the loaded variables
        globals().update(loaded_variables)

#### **Auxiliary Functions**

In [ ]:
def times_train_MG(MG, train, time_period=525600):
    """
    Given a microgrid (MG) and a train, this function returns the corresponding times in the year (or
    time period) when the train is at that microgrid.

    Parameters:
    - MG: The microgrid string (e.g., 'MG4', 'MG5').
    - train: The train string (e.g., 'Train1', 'Train2').
    - time_period: The total time period in minutes (default is 525600 for a full non-leap year).

    Returns:
    - time_at_MG: A numpy array of times in the year (or time period) when the train is at the specified microgrid.
    - time_not_at_MG: A numpy array of times in the year (or time period) when the train is not at the specified microgrid,
      including the times before the start of the train's first cycle.
    """

    # Step 1: Identify times in the train's cycle where it is at the specified MG.
    # `time_arr_cycle` stores indices in the cycle where the train is at the MG.
    time_arr_cycle =  np.where(np.array(TrainAtMG[train]) == MG)[0]

    # Step 2: Retrieve the cycle start time and cycle length for the train.
    cycle_start = TrainCycle[train][0]
    cycle_length = TrainCycle[train][1] - cycle_start
    n_repeats = (time_period - cycle_start) // cycle_length + 1  # Represents how many times the cycle can fit within `time_period`.

    # Step 3: Calculate times in the year (or time period) when the train is at the MG.
    # Using broadcasting, we calculate each cycle repeat's start time (using cycle_start +
    # n_repeats * cycle_length) and add the times in `time_arr_cycle` to get the absolute times in the year.
    n_repeats = (time_period - cycle_start) // cycle_length + 1
    time_arr = cycle_start + np.arange(n_repeats)[:, None] * cycle_length + time_arr_cycle

    # Step 5: Filter times within the time period.
    time_at_MG = time_arr[time_arr < time_period]  #  Filter for times within `time_period` to ensure no overflow.
    time_not_at_MG = np.setdiff1d(np.arange(time_period), time_at_MG)  # By taking the set difference between the entire range and `time_at_MG`, we get `time_not_at_MG`.

    return time_at_MG, time_not_at_MG

In [ ]:
def which_train_at_MG(m, t):
    """
    Determines which train (if any) is at a specified microgrid (MG) at a given time.

    Parameters:
    - m: The microgrid identifier (e.g., 'MG4', 'MG5').
    - t: The time (in minutes) for which we want to check which train is at the microgrid.

    Returns:
    - The train identifier (e.g., 'Train1', 'Train2') if a train is at the microgrid at time t.
      If no train is at the microgrid at that time, it returns False.
    """

    i = False  # Initialize i as False, which will be returned if no train is found at the MG at time t.

    for ii in trains:  # Loop through each train in the train list
        # Step 1: Calculate the corresponding time in the train's cycle
        # The formula computes where the given time 't' falls within the repeating cycle of the train.
        t2TrainCycle = np.mod(t - TrainCycle[ii][0], TrainCycle[ii][1] - TrainCycle[ii][0])

        # Step 2: Check if the train is at the specified microgrid 'm' during that cycle time
        # TrainAtMG[ii] gives the location of the train at each time in its cycle.
        if TrainAtMG[ii][t2TrainCycle] == m:
            i = ii  # If the train is at the microgrid, assign the train's identifier (ii) to 'i'
            break  # Stop checking further trains as we already found the train at the microgrid

    return i  # Return the train identifier or False if no train was found

In [ ]:
def row_idx_train_MG(mg_index, train_index):
    """
    Calculates row index the row index for a given microgrid index and train index.
    """
    return mg_index * len(trains) + train_index

In [ ]:
def times_train_not_at_any_MG(train, time_period=525600):
    """
    Given a train and a time period, return the corresponding times in the year (or time period)
    when the train is not at ANY microgrid.

    Parameters:
    - train: The train string (e.g., 'Train1', 'Train2').
    - time_period: The total time period in minutes (default is 525600 for a full non-leap year).

    Returns:
    - time_not_at_all_MG: A numpy arry of times in the year (or time period) when the train is not at ANY microgrid,
      including the times before the start of the train's first cycle.
    """

    not_at_any_MG = np.arange(time_period)  # Initialization with all times

    for m in MGs:
        time_at_MG, _ = times_train_MG(m, train, time_period)  # Get times train is at a specific MG
        not_at_any_MG = np.setdiff1d(not_at_any_MG, time_at_MG)  # Remove times from the array

    return not_at_any_MG

In [ ]:
def times_MG_with_no_train(MG, time_period=525600):
    """
    Given a microgrid (MG) and a time period, return the corresponding times in the year (or time period)
    when the microgrid has no train.

    Parameters:
    - MG: The microgrid string (e.g., 'MG4', 'MG5').
    - time_period: The total time period in minutes (default is 525600 for a full non-leap year).

    Returns:
    - time_MG_with_no_train: A numpy array of times in the year (or time period) when the microgrid has no train,
      including the times before the start of a train's first cycle.
    """

    time_MG_with_no_train = np.arange(time_period)  # Initialize with all times

    for t in trains:
        time_at_MG, _ = times_train_MG(MG, t, time_period)  # Get a specific train is at the MG
        time_MG_with_no_train = np.setdiff1d(time_MG_with_no_train, time_at_MG)  # Remove times from the array

    return time_MG_with_no_train

In [ ]:
print(times_train_not_at_any_MG('Train1'))

[     0      1      2 ... 525597 525598 525599]


#### **Extract and Set Data**

##### **Data Functions**

In [ ]:
def extract_train_time_data(file_path, MGs, trains, TrainCycle):
    """
    Extracts train data (load, regenerative power, and microgrid locations) for each train over time
    from an Excel file.

    Parameters:
    - file_path: The path to the Excel file containing train cycle data.
    - MGs: A dictionary or list containing microgrid names (e.g., ['MG1', 'MG2', ...]).
    - trains: A list containing the train names (e.g., ['Train1', 'Train2', 'Train3']).
    - TrainCycle: A dictionary containing the start and end time for each train's cycle.

    Returns:
    - TrainLoad: A dictionary where keys are train names, and values are lists of the train load per minute.
    - REG2TrainESS: A dictionary where keys are train names, and values are lists of regenerative power values per minute.
    - TrainAtMG: A dictionary where keys are train names, and values are lists indicating which microgrid (if any) the train is at for each minute.
    """

    # Initialize dictionaries to store train data (load, regenerative power, microgrid location)
    TrainLoad = {train: {} for train in trains}
    REG2TrainESS = {train: {} for train in trains}
    TrainAtMG = {train: {} for train in trains}

    # Define the sheet names dynamically based on train names
    # Assumes the sheets are named in the format 'Train1_cycle', etc.
    train_sheets = [f'{train}_cycle' for train in trains]

    for ind, train in enumerate(TrainLoad):  # Iterate over each train
        sheet = train_sheets[ind]  # Get the corresponding sheet for the train

        # Extract microgrid (MG) location data for the train
        train_at_MG = []
        MG_times = pd.read_excel(file_path, sheet_name=sheet, usecols="I:M", skiprows=4)
        # Each row in the MG_times dataframe corresponds to whether the train is at a particular MG at that minute.

        # For each minute in the cycle, check which MG the train is at, or if it is not at any MG
        cycle_length = TrainCycle[train][1] - TrainCycle[train][0]
        for t in range(cycle_length):
            arr = MG_times.iloc[t].to_numpy()  # Convert row to NumPy array for processing
            train_at_MG.append(MGs[np.argmax(arr)] if np.any(arr) else False)
            # np.argmax returns the index of the MG where the train is at (highest value). If no MG is detected, append False.

        TrainAtMG[train] = train_at_MG  # Store the result in the TrainAtMG dictionary for the current train

        # Extract train load data for each minute in the cycle
        train_load = pd.read_excel(file_path, sheet_name=sheet, usecols="G", skiprows=4)
        TrainLoad[train] = train_load.iloc[0:cycle_length, 0].tolist()  # Convert the column to a list and store it

        # Extract regenerative power data for each minute in the cycle
        regen = pd.read_excel(file_path, sheet_name=sheet, usecols="H", skiprows=4)
        REG2TrainESS[train] = np.abs(regen.iloc[0:cycle_length, 0]).tolist()  # Regenerative power is negated (as input) and stored as a list

    return TrainLoad, REG2TrainESS, TrainAtMG


In [ ]:
def extract_grid_time_data(file_path):
    """
    Extracts the grid price per hour from the Excel file and converts it from cents to dollars per kWh.

    Parameters:
    - file_path: The path to the Excel file containing grid price data.

    Returns:
    - grid_price_hour: A list containing the grid price per hour in dollars per kWh.
    """

    # Read grid price per hour from the Excel file and convert from cents to dollars
    grid_price_hour = pd.read_excel(file_path, sheet_name='gridprice_hour', usecols="C", skiprows=7) / 100

    return grid_price_hour.iloc[:, 0].tolist()  # Convert the DataFrame column to a list


In [ ]:
def extract_PV_time_data(file_path):
    """
    Extracts and interpolates solar power production data from hourly values to minute-level values. Notice that all MGs share the same PV production values.

    Parameters:
    - file_path: The path to the Excel file containing solar power production data.

    Returns:
    - A sparse matrix with minute-level solar power production for the microgrids.
    """

    def interpolate_minute(minute_pv_prod, P_prev, P_next):
        """
        Interpolates solar power production between two consecutive hours to get per-minute values.

        Parameters:
        - minute_pv_prod: List to store interpolated minute-level PV production values.
        - P_prev: The PV production value at the previous hour.
        - P_next: The PV production value at the next hour.

        Returns:
        - minute_pv_prod: Updated list with interpolated minute-level values.
        """
        for minute in range(60):
            P_minute = P_prev + (P_next - P_prev) * (minute / 60)  # Linear interpolation for each minute
            minute_pv_prod.append(P_minute/60)  # Append the interpolated value and adjust it to minute-level
        return minute_pv_prod

    # Step 1: Read hourly PV production data from the Excel file (for two microgrids)
    pv_prod_hour = pd.read_excel(file_path, sheet_name='solar_hour', usecols="C", skiprows=7)

    # Step 2: Add the first row to the end of the dataframe to handle the cyclic nature of the data (yearly repeat)
    pv_prod_hour.loc[len(pv_prod_hour)] = pv_prod_hour.iloc[0]

    # Step 3: Initialize a list to store minute-level PV production values
    pv_prod_minute = []
    zero_minutes = [0] * 60  # A list of zero values for 60 minutes (used when both P_prev and P_next are 0)

    # Step 4: Loop through each hour and interpolate minute-level values
    for hour in range(len(pv_prod_hour) - 1):
        P_prev, P_next = pv_prod_hour.iloc[hour].values[0], pv_prod_hour.iloc[hour + 1].values[0]  # Get PV production for current and next hour
        if not (P_prev.any() or P_next.any()):  # If both values are zero, append zero minutes
            pv_prod_minute.extend(zero_minutes)
        else:  # Otherwise, interpolate between the two hourly values
            pv_prod_minute = interpolate_minute(pv_prod_minute, P_prev, P_next)

    return sparse.csr_matrix(np.array(pv_prod_minute))  # Step 5: Convert the minute-level PV production data to a sparse matrix format for memory efficiency


##### **Define Dataset**

In [ ]:
# Define the file path to the Excel file containing the train and grid data

# Define the list of trains, microgrids (MGs), and MGs with PV for which data will be extracted
file_path = '/content/drive/MyDrive/Colab_Notebooks/Voltify/model_train_data.xlsx'
trains = ['Train1', 'Train2', 'Train3']
MGs = ['MG1', 'MG2', 'MG3', 'MG4', 'MG5']
GRID_MGs = ['MG3', 'MG4', 'MG5']

num_trains = len(trains)
num_MGs = len(MGs)

In [ ]:
# Step 1: Define the train cycle information and time period
# TrainCycle defines the start and end minute of each train's cycle
TrainCycle = {
    'Train1': (60, 60 + 7338),  # Train1 cycle starts at minute 60 (slice 59) and ends after 7338 minutes
    'Train2': (700, 700 + 6944),  # Train2 cycle starts at minute 700 (slice 699) and ends ends after 6944 minutes
    'Train3': (1380, 1380 + 7101)  # Train3 cycle starts at minute 1380 (slice 1379) ends after 7101 minutes
}

# Define the total number of minutes in the year (T) or a shorter time period for testing
T = 24*60*365  # Number of minutes in a year (non-leap year)

In [ ]:
# Step 2: Extract data from the Excel file
# Extract data related to train loads in kWh/minute, regenerative power in kWh/minute, and MG locations for each train
TrainLoad, REG2TrainESS, TrainAtMG = extract_train_time_data(file_path, MGs, trains, TrainCycle)

# Extract grid price in $ for 1kWh per hour, over the entire year
GRIDprice = extract_grid_time_data(file_path)

# Extract solar PV production data and interpolate to minute-level data in kWh/minute/kWp.
PVprod = extract_PV_time_data(file_path)

In [ ]:
# Step 3: Set Train ESS parameters
TrainESSmin = {train: 500 for train in trains}  # Minimum state of charge (SOC) for each train ESS in kWh
TrainESScap = {train: 20000 for train in trains}  # Train ESS capacity (maximum SOC) in kWh
TrainESSchargeMAX = {train: 0.5*TrainESScap[train]/60 for train in trains}  # Max charge rate for each train ESS in kWh/minute
TrainESSdischargeMAX = TrainESSchargeMAX  # Max discharge rate is set to be the same as charge rate
TrainESSchargeLoss = {train: 0.04 for train in trains}  # Charge loss for each train ESS
TrainESSdischargeLoss = TrainESSchargeLoss  # Discharge loss is set to be the same as charge loss

In [ ]:
# Step 4: Set MG ESS parameters
MGESSmin = {MG: 500 for MG in MGs}  # Min SOC for each MG ESS in kWh
MGESSchargeMAX_ratio = {MG: 0.5/60 for MG in MGs}  # Max charge rate ratio for each MG ESS in kWh/minute (MGESSchargeMAX is a variable: MGESSchargeMAX=MGESScap*MGESSchargeMAX_ratio)
MGESSdischargeMAX_ratio = MGESSchargeMAX_ratio  # Max discharge rate is set to be the same as charge rate (MGESSdischargeMAX is a variable: MGESSdischargeMAX=MGESScap*MGESSdischargeMAX_ratio)
MGESSchargeLoss = {MG: 0.04 for MG in MGs}  # Charge loss for each MG ESS
MGESSdischargeLoss = MGESSchargeLoss  # Discharge loss is set to be the same as charge loss

In [ ]:
# Step 5: Initialize state of charge (SOC) for train ESS and MG ESS
# This defines the initial SOC for each train and MG ESS,
TrainESSsocInit = {train: 5000 for train in trains}  # Initial SOC for train ESS in kWh
MGESSsocInit_ratio = {MG: 1 for MG in MGs}  # Initial SOC ratio for MG ESS (MGESSsocInit is a variable: MGESSsocInit=MGESScap*MGESSsocInit_ratio)

In [ ]:
# Step 6: Set grid production for each MG (in KWh) and maximum PV capacity for installation
GRIDprod = {'MG1': 0, 'MG2': 0, 'MG3': 8.3/60, 'MG4': 12/60, 'MG5': 8.3/60}  # Grid power production in kWh/minute
PVcapMAX = {'MG1': 20000, 'MG2': 20000, 'MG3': 20000, 'MG4': 15000, 'MG5': 15000}  # Maximum PV capacity to be installed in kWp
MGESScapMAX = {'MG1': 20000, 'MG2': 20000, 'MG3': 20000, 'MG4': 20000, 'MG5': 20000}  # Maximum MGESS capacity to be installed in kWh

In [ ]:
# Step 7: Calculate regenerative power with respect to max charge rate for Train ESS
# Ensure regenerative power does not exceed the max charge rate of the train ESS
REG2TrainESSs = {
    train: [min(REG2TrainESS[train][t], TrainESSchargeMAX[train]) for t in range(len(REG2TrainESS[train]))]
    for train in trains
}

In [ ]:
# Step 8: Adjust Train ESS capacity to account for difference between regenerative power and train load
# TrainESScap is adjusted to ensure capacity is not exceeded by incoming regenerative power and load
TrainESScap = {
    train: [TrainESScap[train] - np.max([REG2TrainESS[train][t] - TrainLoad[train][t], 0])
            for t in range(len(REG2TrainESS[train]))]
    for train in trains
}

In [ ]:
# Step 9: Set solver type
# GLPK_MI is a solver option to be used for optimization (mixed-integer programming)
solver = 'GLPK_MI'

In [ ]:
# Step 10: Set installation costs, maintenance costs, and coefficients
PVcost = {MG: 1000 for MG in MGs}  # Cost of installing PV in $ per 1kWp
MGESScost = {MG: 120 for MG in MGs}  # Cost of installing MG ESS in $ per 1kWh
CostsCoeff = 0.07  # Coefficient for weighing investment costs (VERIFY)
PVmaintCoeff = {MG: 3 for MG in MGs}  # PV maintenance cost coefficient in $ per 1kWp in one year
MGESSmaintCoeff = {MG: 1000 for MG in MGs}  # MG ESS maintenance cost coefficient in $ per 1kWh in one year

# Set maximum power transfer from MG to train
MG2TrainMAX = {MG: TrainESSchargeMAX for MG in MGs}  # Max power transfer from MG to train in kWh/minute

In [ ]:
save_variables('/content/drive/MyDrive/Colab_Notebooks/Voltify/dataset.pkl')

#### **Run Problem**

In [ ]:
# Load variables from the file
load_variables('/content/drive/MyDrive/Colab_Notebooks/Voltify/dataset.pkl')

In [ ]:
T=1000

In [ ]:
print('Solving the MG System Planning MLIP Problem using CVXPY and the', solver, 'Solver')


# Initialize the total cost variables and constraints list
GridCost = 0            # Tracks cost of energy drawn from the grid across all MGs
MaintenanceCost = 0     # Tracks maintenance cost for PV and MG ESS systems across all MGs
Investment = 0          # Tracks capital investment for PV and MG ESS installations
constraints = []        # List to hold all constraints for the optimization problem

# Define Train Decision Variables
TrainESS2TrainLoad = cp.Variable((num_trains, T), nonneg=True)      # Power sent from Train ESS to Train Load, defined for each train at each time step
TrainESSsoc = cp.Variable((num_trains, T), nonneg=True)             # SoC of Train ESS for each train at each time step
                                                                    # NOTE: The variable TrainESSsoc assumes that the start time of each train is at least 1, placing the initial SoC at index 0.
                                                                    #       If the start time is at time 0, TrainESSsoc would need T+1 columns.

# Define MG Decision Variables
PV2MGESS = cp.Variable((num_MGs, T), nonneg=True)           # PV power production going to MG ESS for each MG at each time step
GRID2MGESS = cp.Variable((num_MGs, T), nonneg=True)         # Grid power going to MG ESS for each MG at each time step
MGESSsoc = cp.Variable((num_MGs, T+1), nonneg=True)         # SoC of MG ESS for each MG at each time step, starting from time 0 (with an additional time step for initial SoC)
PVcap = cp.Variable(num_MGs, nonneg=True)                   # PV capacity in kWp for each MG
MGESScap = cp.Variable(num_MGs, nonneg=True)                # MG ESS capacity in kWh for each MG
#MGESSsocInit = cp.Variable(num_MGs, nonneg=True)            # MG ESS initial SoC for each MG
MGESSinstall = cp.Variable(num_MGs, boolean=True)           # Binary variable indicating whether MG ESS is installed at each MG (1 = installed, 0 = not installed)
MGESSchargeMAX = cp.Variable(num_MGs, nonneg=True)          # Maximum allowable charge rate for each MG ESS in kWh/min
MGESSdischargeMAX = cp.Variable(num_MGs, nonneg=True)       # Maximum allowable discharge rate for each MG ESS in kWh/min

# Define Interaction Variables for Train and MG
# The following variables represent the power exchanges between trains and MGs, organized by row:
# Each row corresponds to an MG-Train pair (e.g., row 0 represents MG1-Train1, row 1 MG1-Train2, etc.)
PV2Train = cp.Variable((num_MGs*num_trains, T), nonneg=True)        # PV to train
GRID2Train = cp.Variable((num_MGs*num_trains, T), nonneg=True)      # Grid to train
MGESS2Train = cp.Variable((num_MGs*num_trains, T), nonneg=True)     # MG ESS to Train
MG2TrainESS = cp.Variable((num_MGs*num_trains, T), nonneg=True)     # MG to Train ESS


# Constraint initialization (not time-dependent)
for m, MG in enumerate(MGs):
    # Initialize the MGESS SoC at time 0 based on the initial SoC ratio and the MGESS capacity
    constraints += [MGESSsoc[m, 0] == MGESSsocInit_ratio[MG] * MGESScap[m],

                    # Ensure minimum SoC if MGESS is installed:
                    # If MGESSinstall[m] == 1, enforce SoC >= MGESSmin[MG]. If MGESSinstall[m] == 0, SoC is forced to be 0.
                    MGESSmin[MG] * MGESSinstall[m] <= MGESSsoc[m, 0],

                    # MGESS capacity constraint: Only non-zero if MGESS is installed (MGESSinstall[m] == 1),
                    # bounded by the maximum allowed MGESS capacity to be installed (MGESScapMAX[MG]).
                    MGESScap[m] <= MGESSinstall[m] * MGESScapMAX[MG],

                    # Define the maximum MGESS charging and discharging rates as a function of the capacity
                    MGESSchargeMAX[m] == MGESSchargeMAX_ratio[MG] * MGESScap[m],
                    MGESSdischargeMAX[m] == MGESSdischargeMAX_ratio[MG] * MGESScap[m],

                    # PV capacity constraint: Ensure installed PV capacity does not exceed maximum allowed capacity to be installed for each MG
                    PVcap[m] <= PVcapMAX[MG]]

    if MG not in GRID_MGs:  # If the current MG does not have GRID capabilities, set related variables to 0 for all time steps
        constraints += [GRID2Train[np.array([m * num_trains + i for i in range(num_trains)]), :] == 0,  # Zero out rows for all trains at this MG
                        GRID2MGESS[m, :] == 0]  # Set GRID2MGESS to 0 for this MG


# Loop through each train and define constraints over all time periods
for i, train in enumerate(trains):
    trainStartTime = TrainCycle[train][0]  # Start time of the train's first cycle
    trainCycleLen = TrainCycle[train][1] - trainStartTime  # Length of the train's cycle
    row_indxs = [i + m * num_trains for m in range(num_MGs)]  # Row indices for each MG corresponding to the train

    # Set initial conditions for train ESS
    constraints += [TrainESS2TrainLoad[i, :trainStartTime] == 0,  # No discharge before the train start time
                    TrainESSsoc[i, :trainStartTime] == TrainESSsocInit[train],  # SoC remains at initial value before the train start time

                    # For each MG associated with the train, interactions (PV, Grid, MGESS) are inactive before train start
                    PV2Train[row_indxs, :trainStartTime] == 0,
                    GRID2Train[row_indxs, :trainStartTime] == 0,
                    MGESS2Train[row_indxs, :trainStartTime] == 0,
                    MG2TrainESS[row_indxs, :trainStartTime] == 0]


    # Define discharge and SoC constraints for Train ESS during active time periods (from start time to T):
    t2TrainCycle = np.mod(np.arange(trainStartTime, T) - trainStartTime, trainCycleLen)  # Define time in terms of the relative position within the train cycle
    TrainESScap_cycle = np.tile(TrainESScap[train], int(np.ceil(len(t2TrainCycle) / trainCycleLen)))[:len(t2TrainCycle)]  # Array for ESS capacity based on cycle times (for all cycles)
    # Equivalent to `np.array([TrainESScap[train][t_cycle] for t_cycle in t2TrainCycle])` with no loop

    if trainStartTime < T:  # This statement is False only for test cases with small T values.
        constraints += [TrainESS2TrainLoad[i, trainStartTime:] <= TrainESSdischargeMAX[train],  # Ensure TrainESS2TrainLoad does not exceed max discharge rate.
                        TrainESSmin[train] <= TrainESSsoc[i, trainStartTime:],  # Maintain SoC within min and capacity limits.
                        TrainESSsoc[i, trainStartTime:] <= TrainESScap_cycle]


    # Times when the train is not at any MG
    t_no_MG = np.setdiff1d(times_train_not_at_any_MG(train, T), np.arange(trainStartTime))
    if len(t_no_MG) > 0:
        t2TrainCycle_no_MG = np.mod(np.array(t_no_MG) - trainStartTime, trainCycleLen)  # Calculate time within the train cycle for times without MG interaction
        # Precompute load and regenerative power for these times in the cycle
        TrainLoad_no_MG = np.take(TrainLoad[train], t2TrainCycle_no_MG)
        REG2TrainESS_no_MG = np.take(REG2TrainESS[train], t2TrainCycle_no_MG)

        # Constraints for train operation when not at any MG
        constraints += [TrainLoad_no_MG == TrainESS2TrainLoad[i, t_no_MG] * (1 - TrainESSdischargeLoss[train]),  # Train load must match ESS discharge rate, adjusted for discharge loss

                        # SoC of the train ESS is updated based on regenerative power and discharge activity
                        TrainESSsoc[i, t_no_MG] == TrainESSsoc[i, t_no_MG - 1] + REG2TrainESS_no_MG * (1 - TrainESSchargeLoss[train]) - TrainESS2TrainLoad[i, t_no_MG]]


    for m, MG in enumerate(MGs):
        t_MG, t_not_MG = times_train_MG(MG, train, T)  # Identify the times when the train is at and not at the given MG
        t_not_MG = np.setdiff1d(t_not_MG, np.arange(trainStartTime))  # Only include times the train is not at the given MG after the start of its first cycle (times before are already handled)
        row_idx = row_idx_train_MG(m, i)  # Row index for this MG and train combination

        if len(t_MG) > 0:  # If the train is at this MG during certain times
            # Calculate time within the train cycle for times at this MG
            t2TrainCycle_MG = np.mod(np.array(t_MG) - trainStartTime, trainCycleLen)
            # Precompute load and regenerative power for times at this MG based on cycle times (for all cycles)
            TrainLoad_MG = np.take(TrainLoad[train], t2TrainCycle_MG)
            REG2TrainESS_MG = np.take(REG2TrainESS[train], t2TrainCycle_MG)

            # Add constraints for when the train is at the MG:
            # Total train load must equal incoming power minus outgoing power, adjusted by losses
            constraints += [TrainLoad_MG == PV2Train[row_idx, t_MG] + GRID2Train[row_idx, t_MG] + MGESS2Train[row_idx, t_MG] * (1 - MGESSdischargeLoss[MG]) - MG2TrainESS[row_idx, t_MG] + TrainESS2TrainLoad[i, t_MG] * (1 - TrainESSdischargeLoss[train]),

                            # Power transfer must not exceed MG-to-train max capacity
                            PV2Train[row_idx, t_MG] + GRID2Train[row_idx, t_MG] + MGESS2Train[row_idx, t_MG] <= MG2TrainMAX[MG][train],

                            # Charging rates for MG-to-train and regenerative sources must not exceed max charge rate
                            MG2TrainESS[row_idx, t_MG] + REG2TrainESS_MG <= TrainESSchargeMAX[train],

                            # Net inflow to the train ESS must be non-negative
                            PV2Train[row_idx, t_MG] + GRID2Train[row_idx, t_MG] + MGESS2Train[row_idx, t_MG] - MG2TrainESS[row_idx, t_MG] >= 0,

                            # Train ESS SoC updates based on power flows, charge losses, and discharge activity
                            TrainESSsoc[i, np.array(t_MG)] == TrainESSsoc[i, np.array(t_MG)-1] + (MG2TrainESS[row_idx, t_MG] + REG2TrainESS_MG) * (1 - TrainESSchargeLoss[train]) - TrainESS2TrainLoad[i, t_MG]]

        if len(t_not_MG) > 0:  # If the train is not at this MG during certain times
            # Set power-related variables to zero for times when the train is not at the MG
            constraints += [PV2Train[row_idx, t_not_MG] == 0,
                            GRID2Train[row_idx, t_not_MG] == 0,
                            MGESS2Train[row_idx, t_not_MG] == 0,
                            MG2TrainESS[row_idx, t_not_MG] == 0]



# Generate an array that maps each minute of the year to its respective hour, allowing easy access to hourly grid prices
t2YearHour = np.arange(T) // 60  # Array mapping each minute in the year to its corresponding hour index
GRIDprice_minute = np.take(GRIDprice, t2YearHour)  # Maps each minute to its grid price based on the hour index

# Initialize costs associated with the grid, maintenance, and investment
for m, MG in enumerate(MGs):
    # Accumulate grid, maintenance, and investment costs for the current microgrid
    GridCost += cp.sum(cp.multiply(GRID2MGESS[m, :], GRIDprice_minute))
    MaintenanceCost += PVmaintCoeff[MG]*PVcap[m] + MGESSmaintCoeff[MG]*MGESScap[m]
    Investment += PVcost[MG]*PVcap[m] + MGESScost[MG]*MGESScap[m]


    # Set constraints for MGESS charging and SoC limits
    constraints += [PV2MGESS[m, :] + GRID2MGESS[m, :] <= MGESSchargeMAX[m],  # Total power input to MGESS cannot exceed max charge rate
                    MGESSmin[MG] * MGESSinstall[m] <= MGESSsoc[m, 1:],  # Enforce minimum SoC if MGESS is installed; SoC must be at least MGESSmin[MG]
                    MGESSsoc[m, 1:] <= MGESScap[m]]  # SoC cannot exceed MGESS capacity


    # When no train is connected to this MG, manage PV and GRID constraints
    t_no_train = times_MG_with_no_train(MG, T)  # Times when MG has no train connected
    if len(t_no_train) > 0:
        PVprod_dense = np.reshape(np.array(PVprod[0, t_no_train].todense()), -1)  # Flatten PV production values for these times
        constraints += [MGESSsoc[m, t_no_train + 1] == MGESSsoc[m, t_no_train] + (PV2MGESS[m, t_no_train] + GRID2MGESS[m, t_no_train]) * (1 - MGESSchargeLoss[MG]),  # MGESS SoC transition
                    PV2MGESS[m, t_no_train] <= PVprod_dense * PVcap[m],  # PV input to MGESS cannot exceed production
                    GRID2MGESS[m, t_no_train] <= GRIDprod[MG]]  # Grid input to MGESS limited by GRID capacity


    # Loop over each train to handle times when it may be connected to this MG (assumes that no more than one train is connected to a given MG at any time)
    for i, train in enumerate(trains):
        t_MG, _ = times_train_MG(MG, train, T)  # Get times when train is at this MG
        row_idx = row_idx_train_MG(m, i)  # Calculate row index for this MG-train pair

        if len(t_MG) > 0:  # Apply constraints if the train is connected to this MG during certain times (the case where the train is not connected has already been handled)
            PVprod_dense = np.reshape(np.array(PVprod[0, t_MG].todense()), -1)  # Flatten PV production values for these times
            t2YearHour_MG = np.array(t_MG) // 60  # Map each minute to the corresponding hour index
            GRIDprice_at_MG = np.take(GRIDprice, t2YearHour_MG)  # Hourly grid prices for relevant minutes
            GridCost += cp.sum(cp.multiply(GRID2Train[row_idx, t_MG], GRIDprice_at_MG))  # Accumulate grid cost for train at this MG

             # Enforce constraints on the energy flow between MGESS, grid, and train ESS when train is at MG
            constraints += [MGESS2Train[row_idx, t_MG] <= MGESSdischargeMAX[m],  # Discharge from MGESS to train is within limit
                            MGESSsoc[m, np.array(t_MG) + 1] == MGESSsoc[m, t_MG] + (PV2MGESS[m, t_MG] + GRID2MGESS[m, t_MG]) * (1 - MGESSchargeLoss[MG]) - MGESS2Train[row_idx, t_MG],  # MGESS SoC transition
                            PV2MGESS[m, t_MG] + PV2Train[row_idx, t_MG] <= PVprod_dense * PVcap[m],  # Total PV input limited by production
                            GRID2MGESS[m, t_MG] + GRID2Train[row_idx, t_MG] <= GRIDprod[MG]]  # Total grid input limited by capacity



# Objective function: Minimize investment, grid cost, and maintenance cost
objective = cp.Minimize(Investment + CostsCoeff * (GridCost + MaintenanceCost))

# Problem definition
problem = cp.Problem(objective, constraints)

# Solve the problem
problem.solve(solver=solver, verbose=True)

# Solution status
print(f"-------------------------------------------------------------------------------")
print(f"                                      Done                                     ")
print(f"-------------------------------------------------------------------------------")
print("Solution status: ", problem.status)



Solving the MG System Planning MLIP Problem using CVXPY and the GLPK_MI Solver
                                     CVXPY                                     
                                     v1.5.3                                    
(CVXPY) Oct 27 12:49:24 PM: Your problem has 81030 variables, 107750 constraints, and 0 parameters.
(CVXPY) Oct 27 12:49:24 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Oct 27 12:49:24 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Oct 27 12:49:24 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Oct 27 12:49:24 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-----------------------------------------------------------------

In [ ]:
"Print the optimal values and solutions"

print(f'===============================================================================')

print(f"Total cost: {round(problem.value, 2)}")
print(f"Grid Cost: {round(GridCost.value, 2)}")
print(f"Maintenance Cost: {round(MaintenanceCost.value, 2)}")
print(f"Investment: {round(Investment.value, 2)}")

print("PVcap (Installed capacity of PV at each MG): ")
for m, MG in enumerate(MGs):
    pv_cap_val = round(PVcap[m].value, 2)
    if pv_cap_val > 0:
        print(f"     {MG}: {pv_cap_val}")
    else:
        print(f"     {MG}: No capacity installed")

print("MGESScap (Installed capacity of MGESS at each MG): ")
for m, MG in enumerate(MGs):
    mges_cap_val = round(MGESScap[m].value, 2)
    if mges_cap_val > 0:
        print(f"     {MG}: {mges_cap_val}")
    else:
        print(f"     {MG}: No capacity installed")

# Display MGESS installation status
# print("MGESSinstall (Installation status of MGESS at each MG): ")
# for m, MG in enumerate(MGs):
#     print(f"     {MG}: {'Installed' if MGESSinstall[m].value > 0 else 'Not Installed'}")

# Helper function to display non-zero entries with aligned format
def display_nonzero_matrix(matrix, label, ids):
    all_zero = True  # Flag to check if entire matrix is zero
    for i, ID in enumerate(ids):
        # Find non-zero time indices and their values
        non_zero_times = [t for t in range(matrix.shape[1]) if matrix[i, t].value != 0]
        if non_zero_times:
            all_zero = False
            non_zero_values = [round(matrix[i, t].value, 2) for t in non_zero_times]

            # Format strings for alignment
            times_str = " ".join(f"{str(t):<9}" for t in non_zero_times)  # Left-align with 9 spaces per entry
            values_str = " ".join(f"{str(v):<9}" for v in non_zero_values)  # Match space for values

            print(f"     {ID}:")
            print(f"         Times : {times_str}")
            print(f"         Values: {values_str}")
    if all_zero:
        print("     All entries are zero")

# Display matrices with pairs for MGs and trains
mg_train_pairs = [(MG, train) for MG in MGs for train in trains]
print("PV2Train (PV energy supplied to trains at each MG):")
display_nonzero_matrix(PV2Train, "PV2Train", [f"{MG}-{train}" for MG, train in mg_train_pairs])

print("GRID2Train (Grid energy supplied to trains at each MG):")
display_nonzero_matrix(GRID2Train, "GRID2Train", [f"{MG}-{train}" for MG, train in mg_train_pairs])

print("MGESS2Train (MGESS energy supplied to trains at each MG):")
display_nonzero_matrix(MGESS2Train, "MGESS2Train", [f"{MG}-{train}" for MG, train in mg_train_pairs])

print("MG2TrainESS (Energy transferred from MG to Train ESS at each MG):")
display_nonzero_matrix(MG2TrainESS, "MG2TrainESS", [f"{MG}-{train}" for MG, train in mg_train_pairs])

# Display state of charge for Train ESS (only train ID) and MGESS (only MG ID)
print("TrainESSsoc (SoC of Train ESS):")
display_nonzero_matrix(TrainESSsoc, "TrainESSsoc", trains)

print("MGESSsoc (SoC of MGESS at each MG):")
display_nonzero_matrix(MGESSsoc, "MGESSsoc", MGs)

print("TrainESS2TrainLoad (Energy from Train ESS to Train Load):")
display_nonzero_matrix(TrainESS2TrainLoad, "TrainESS2TrainLoad", trains)

Total cost: 1818311.34
Grid Cost: 6.77
Maintenance Cost: 9570057.21
Investment: 1148406.86
PVcap (Installed capacity of PV at each MG): 
     MG1: No capacity installed
     MG2: No capacity installed
     MG3: No capacity installed
     MG4: No capacity installed
     MG5: No capacity installed
MGESScap (Installed capacity of MGESS at each MG): 
     MG1: No capacity installed
     MG2: 1969.75
     MG3: 7600.31
     MG4: No capacity installed
     MG5: No capacity installed
PV2Train (PV energy supplied to trains at each MG):
     All entries are zero
GRID2Train (Grid energy supplied to trains at each MG):
     MG3-Train1:
         Times : 60        61        62        63        64        65        66        67        68        69        70        71        72        73        74        75        76        77        78        79        80        81        82        83        84        85        86        87        88        89        90        91        92        93        94        9